In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error
from sklearn.linear_model import LinearRegression, RidgeCV

from models.tft import TemporalFusionTransformer
from models.tcn import TemporalConvolutionalNetwork
from models.hfm import HybridForecastingModel
from models.dcf import DistributionalConditionalForecast

from loader import train_loader, test_loader, val_loader, scaler

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
tft = TemporalFusionTransformer(
	seq_input_dim=2, 
	cal_input_dim=18, 
	d_model=64, 
	nhead=4, 
	num_layers=2, 
	output_dim=2, 
	dropout=0.1
)

tft.to(device)
checkpoint = torch.load("checkpoints/TFT.pth", map_location=device)
tft.load_state_dict(checkpoint["model_state_dict"])
tft.eval()

tcn = TemporalConvolutionalNetwork(
	seq_input_dim=2, 
	cal_input_dim=18, 
	d_model=64, 
	nhead=4, 
	num_layers=2, 
	tcn_channels=[64, 64, 64],
	kernel_size=3, 
	dropout=0.1, 
	output_dim=2,
)

tcn.to(device)
checkpoint = torch.load("checkpoints/TCN.pth", map_location=device)
tcn.load_state_dict(checkpoint["model_state_dict"])
tcn.eval()

hfm = HybridForecastingModel(
	window_size=30,
	num_series=2,
	static_dim=18,
	d_model=64,
	nhead=4,
	num_layers_transformer=2,
    n_blocks_nhits=3,
	nhits_hidden_dim=128, 
	nhits_n_layers=3, 
	deepar_num_layers=2,
    dropout=0.1, 
	output_dim=2
)

hfm.to(device)
checkpoint = torch.load("checkpoints/HFM.pth", map_location=device)
hfm.load_state_dict(checkpoint["model_state_dict"])
hfm.eval()

dcf = DistributionalConditionalForecast(
    window_size=30, 
	num_series=2, 
	static_dim=18,
    latent_dim=32, 
	hidden_dim=128,
    dropout=0.1, 
	output_dim=2
)

dcf.to(device)
checkpoint = torch.load("checkpoints/DCF.pth", map_location=device)
dcf.load_state_dict(checkpoint["model_state_dict"])
dcf.eval()

print("Models loaded successfully!")

Models loaded successfully!


In [4]:
# Hàm thu thập dự báo từ từng mô hình trên một DataLoader
def get_model_predictions(loader, models, device):
    # models: dict chứa các mô hình với key là tên mô hình
    # Mỗi mô hình trả về tensor shape (batch, 2) dự báo
    preds = {name: [] for name in models.keys()}
    actuals = []
    with torch.no_grad():
        for x_seq, x_cal, y in loader:
            x_seq = x_seq.to(device)
            x_cal = x_cal.to(device)
            # Thu thập dự báo của mỗi mô hình
            # TFT, TCN, HFM: trả về (batch,2)
            preds["tft"].append(models["tft"](x_seq, x_cal).cpu().numpy())
            preds["tcn"].append(models["tcn"](x_seq, x_cal).cpu().numpy())
            preds["hfm"].append(models["hfm"](x_seq, x_cal).cpu().numpy())
            # DCF: trả về (batch,4) => lấy phần μ (first 2)
            out_dcf, _, _ = models["dcf"](x_seq, x_cal)
            preds["dcf"].append(out_dcf[:, :2].cpu().numpy())
            actuals.append(y.cpu().numpy())
    # Nối các mảng theo batch
    for key in preds:
        preds[key] = np.concatenate(preds[key], axis=0)
    actuals = np.concatenate(actuals, axis=0)
    return preds, actuals

# Thu thập dự báo trên tập validation
models = {"tft": tft, "tcn": tcn, "hfm": hfm, "dcf": dcf}
val_preds, val_actuals = get_model_predictions(val_loader, models, device)

In [5]:
X_val = np.concatenate([
    val_preds["tft"],
    val_preds["tcn"],
    val_preds["hfm"],
    val_preds["dcf"]
], axis=1)  # shape (n_val, 8)
y_val = val_actuals  # shape (n_val, 2)

In [6]:
meta_model = LinearRegression()
meta_model.fit(X_val, y_val)

LinearRegression()

In [7]:
test_preds, test_actuals = get_model_predictions(test_loader, models, device)
X_test = np.concatenate([
    test_preds["tft"],
    test_preds["tcn"],
    test_preds["hfm"],
    test_preds["dcf"]
], axis=1)  # shape (n_test, 8)
y_test = test_actuals  # shape (n_test, 2)

In [8]:
# Dự báo ensemble bằng meta-model
ensemble_preds = meta_model.predict(X_test)  # (n_test, 2)

# Nếu cần inverse transform về thang đo ban đầu
ensemble_preds_inv = scaler.inverse_transform(ensemble_preds)
y_test_inv = scaler.inverse_transform(y_test)

r2 = r2_score(y_test_inv, ensemble_preds_inv)
mape = mean_absolute_percentage_error(y_test_inv, ensemble_preds_inv)
rmse = np.sqrt(mean_squared_error(y_test_inv, ensemble_preds_inv))

print(f"Ensemble (Linear Regression) Test R-squared: {r2:.4f}")
print(f"Ensemble (Linear Regression) Test MAPE: {mape:.4f}")
print(f"Ensemble (Linear Regression) Test RMSE: {rmse:.4f}")

Ensemble (Linear Regression) Test R-squared: 0.9708
Ensemble (Linear Regression) Test MAPE: 0.2563
Ensemble (Linear Regression) Test RMSE: 119236.0938


In [9]:
r2_units = r2_score(y_test_inv[:, 0], ensemble_preds_inv[:, 0])
mape_units = mean_absolute_percentage_error(y_test_inv[:, 0], ensemble_preds_inv[:, 0])
rmse_units = np.sqrt(mean_squared_error(y_test_inv[:, 0], ensemble_preds_inv[:, 0]))
print(f"Ensemble (Linear Regression) Test R-squared (Units): {r2_units:.4f}")
print(f"Ensemble (Linear Regression) Test MAPE (Units): {mape_units:.4f}")
print(f"Ensemble (Linear Regression) Test RMSE (Units): {rmse_units:.4f}")

r2_revenue = r2_score(y_test_inv[:, 1], ensemble_preds_inv[:, 1])
mape_revenue = mean_absolute_percentage_error(y_test_inv[:, 1], ensemble_preds_inv[:, 1])
rmse_revenue = np.sqrt(mean_squared_error(y_test_inv[:, 1], ensemble_preds_inv[:, 1]))
print(f"Ensemble (Linear Regression) Test R-squared (Revenue): {r2_revenue:.4f}")
print(f"Ensemble (Linear Regression) Test MAPE (Revenue): {mape_revenue:.4f}")
print(f"Ensemble (Linear Regression) Test RMSE (Revenue): {rmse_revenue:.4f}")

Ensemble (Linear Regression) Test R-squared (Units): 0.9691
Ensemble (Linear Regression) Test MAPE (Units): 0.2714
Ensemble (Linear Regression) Test RMSE (Units): 22.7556
Ensemble (Linear Regression) Test R-squared (Revenue): 0.9726
Ensemble (Linear Regression) Test MAPE (Revenue): 0.2412
Ensemble (Linear Regression) Test RMSE (Revenue): 168625.2969


In [10]:
# Hàm thu thập dự báo từ từng mô hình trên một DataLoader
def get_meta_features(loader, models, device):
    preds = {"tft": [], "tcn": [], "hfm": [], "dcf_mu": [], "dcf_lower": [], "dcf_upper": []}
    static_feats = []  # từ x_cal của mỗi sample
    actuals = []
    with torch.no_grad():
        for x_seq, x_cal, y in loader:
            x_seq = x_seq.to(device)
            x_cal = x_cal.to(device)
            # Thu thập dự báo của mỗi mô hình
            pred_tft = models["tft"](x_seq, x_cal)            # (batch, 2)
            pred_tcn = models["tcn"](x_seq, x_cal)            # (batch, 2)
            pred_hfm = models["hfm"](x_seq, x_cal)            # (batch, 2)
            out_dcf, _, _ = models["dcf"](x_seq, x_cal)       # (batch, 4) => [μ, logvar] cho 2 biến
            # Tách ra μ và logvar cho DCF
            mu_dcf = out_dcf[:, :2]  # (batch, 2)
            logvar_dcf = out_dcf[:, 2:]  # (batch, 2)
            sigma_dcf = torch.exp(0.5 * logvar_dcf)
            lower_dcf = mu_dcf - 1.96 * sigma_dcf
            upper_dcf = mu_dcf + 1.96 * sigma_dcf

            preds["tft"].append(pred_tft.cpu().numpy())
            preds["tcn"].append(pred_tcn.cpu().numpy())
            preds["hfm"].append(pred_hfm.cpu().numpy())
            preds["dcf_mu"].append(mu_dcf.cpu().numpy())
            preds["dcf_lower"].append(lower_dcf.cpu().numpy())
            preds["dcf_upper"].append(upper_dcf.cpu().numpy())
            static_feats.append(x_cal.cpu().numpy())
            actuals.append(y.cpu().numpy())
    
    for key in preds:
        preds[key] = np.concatenate(preds[key], axis=0)
    static_feats = np.concatenate(static_feats, axis=0)
    actuals = np.concatenate(actuals, axis=0)
    
    # Ghép các đặc trưng theo thứ tự: TFT, TCN, HFM, DCF μ, DCF lower, DCF upper, static features
    meta_features = np.concatenate([
        preds["tft"],
        preds["tcn"],
        preds["hfm"],
        preds["dcf_mu"],
        preds["dcf_lower"],
        preds["dcf_upper"],
        static_feats
    ], axis=1)
    
    return meta_features, actuals

# Thu thập meta-features trên tập validation
models = {"tft": tft, "tcn": tcn, "hfm": hfm, "dcf": dcf}
X_val = None
y_val = None
X_val, y_val = get_meta_features(val_loader, models, device)  

# Thu thập meta-features trên tập test
X_test, y_test = get_meta_features(test_loader, models, device)

X_meta_val = X_val  
y_val_orig = scaler.inverse_transform(y_val)  
X_meta_test = X_test
y_test_orig = scaler.inverse_transform(y_test)

# Huấn luyện meta-model sử dụng RidgeCV
# Sử dụng các giá trị alpha từ một dải ví dụ
alphas = np.logspace(-3, 3, 7)
meta_model = RidgeCV(alphas=alphas, scoring='neg_mean_squared_error')
meta_model.fit(X_meta_val, y_val_orig)

# Dự báo trên tập test
meta_preds = meta_model.predict(X_meta_test)  # (n_test, 2)

r2 = r2_score(y_test_orig, meta_preds)
mape = mean_absolute_percentage_error(y_test_orig, meta_preds)
rmse = np.sqrt(mean_squared_error(y_test_orig, meta_preds))
print(f"Ensemble (Ridge) Test R-squared: {r2:.4f}")
print(f"Ensemble (Ridge) Test MAPE: {mape:.4f}")
print(f"Ensemble (Ridge) Test RMSE: {rmse:.4f}")

Ensemble (Ridge) Test R-squared: 0.9740
Ensemble (Ridge) Test MAPE: 0.2433
Ensemble (Ridge) Test RMSE: 118243.1877


In [11]:
# Tính chỉ số trên Units và Revenue 

# Units
y_test_units = y_test_orig[:, 0]
meta_preds_units = meta_preds[:, 0]
r2_units = r2_score(y_test_units, meta_preds_units)
mape_units = mean_absolute_percentage_error(y_test_units, meta_preds_units)
rmse_units = np.sqrt(mean_squared_error(y_test_units, meta_preds_units))

# Revenue
y_test_revenue = y_test_orig[:, 1]
meta_preds_revenue = meta_preds[:, 1]
r2_revenue = r2_score(y_test_revenue, meta_preds_revenue)

mape_revenue = mean_absolute_percentage_error(y_test_revenue, meta_preds_revenue)
rmse_revenue = np.sqrt(mean_squared_error(y_test_revenue, meta_preds_revenue))

print(f"Ensemble (Ridge) Test R-squared (Units): {r2_units:.4f}")
print(f"Ensemble (Ridge) Test MAPE (Units): {mape_units:.4f}")
print(f"Ensemble (Ridge) Test RMSE (Units): {rmse_units:.4f}")
print(f"Ensemble (Ridge) Test R-squared (Revenue): {r2_revenue:.4f}")
print(f"Ensemble (Ridge) Test MAPE (Revenue): {mape_revenue:.4f}")
print(f"Ensemble (Ridge) Test RMSE (Revenue): {rmse_revenue:.4f}")

Ensemble (Ridge) Test R-squared (Units): 0.9750
Ensemble (Ridge) Test MAPE (Units): 0.2430
Ensemble (Ridge) Test RMSE (Units): 20.4552
Ensemble (Ridge) Test R-squared (Revenue): 0.9731
Ensemble (Ridge) Test MAPE (Revenue): 0.2436
Ensemble (Ridge) Test RMSE (Revenue): 167221.1184
